# Lista 4 de Otimização Inteira - Matheurísticas

Autor: Diego Ascânio Santos

**Preâmbulo**

In [106]:
%pip install numpy
%pip install scipy
%load_ext autoreload
%autoreload 2

import numpy as np

def repr_constraints(A, b, variables_names : list | str):
    if type(variables_names) == str:
        variables_names = variables_names.split(',')
    result = ''
    for i, constraint in enumerate(A):
        variables = []
        for j, coefficient in enumerate(constraint):
            if coefficient:
                variables.append(
                    f'{coefficient:+.2f} {variables_names[j]}'
                )
        result += ' '.join(variables) + ' = ' + f'{b[i]:.2f}\n'
    return result

def repr_objective_function(c, variables_names : list | str):
    if type(variables_names) == str:
        variables_names = variables_names.split(',')
    variables = []
    for j, coefficient in enumerate(c):
        if coefficient:
            variables.append(
                f'{coefficient:+.2f} {variables_names[j]}'
            )
    result = 'z = min ' + ' '.join(variables) + '\n'
    return result

def repr_LP_problem(A, b, c, variables_names):
    return repr_objective_function(
        c, variables_names
    ) + '\nSubject to:\n' + repr_constraints(
        A, b, variables_names
    )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Exercício 1

In [110]:
from formulacao_bin_packing_lista_4_exercicio_1 import A, b, c, variables_names
from rens import rens, resolve_subproblema_inteiro, resolve_PPL
from rins import rins

### letra a) - RENS

In [111]:
factivel, z, x, F, x_F, I, x_I, A_prime, b_prime, c_prime = rens(A, b, c)

print('Ao executar a relaxação linear do RENS, as variáveis fracionárias e suas respectivas frações são: ')

for j, v in zip(F, x_F):
    print(f'{variables_names.split(',')[j]} = {v:.2f}')

print('O vetor solução de caixas para a construção linearmente relaxada é: ')
_, x_lr, z_lr = resolve_PPL(A, b, c)
print(f'y = {x_lr[0:5]}')

print('O vetor solução de itens em cada caixa, linearmente relaxado, é: ')
result = ''
for i in range(5):
    result += '[ '
    for j in range(5):
        result += f'{x_lr[5 + 5*i + j]:+.2f} '
    result += ']\n'
print(result)

new_variables_names = variables_names + ',' + ','.join(f'n_{i+1:02d}' for i in range(A_prime.shape[1] - A.shape[1]))

print()

print('Todas as demais variáveis foram inteiras e portanto terão seus valores afixados')
print('Isso produz o seguinte subprograma inteiro para a segunda etapa do RENS:\n')
print(
    repr_LP_problem(A_prime, b_prime, c_prime, new_variables_names)
)

print('O resultado da resolução do subprograma inteiro após a aplicação do RENS é: ')

print(f'y = {x[0:5]}')

print('O vetor solução de itens em cada caixa do subprograma inteiro é: ')
result = ''
for i in range(5):
    result += '[ '
    for j in range(5):
        result += f'{x[5 + 5*i + j]:+.2f} '
    result += ']\n'
print(result)

Ao executar a relaxação linear do RENS, as variáveis fracionárias e suas respectivas frações são: 
x_4_2 = 0.75
x_4_3 = 0.25
O vetor solução de caixas para a construção linearmente relaxada é: 
y = [0. 1. 1. 0. 0.]
O vetor solução de itens em cada caixa, linearmente relaxado, é: 
[ +0.00 +0.00 +1.00 -0.00 +0.00 ]
[ +0.00 +1.00 +0.00 +0.00 +0.00 ]
[ +0.00 +0.00 +1.00 +0.00 +0.00 ]
[ +0.00 +0.75 +0.25 +0.00 -0.00 ]
[ -0.00 +1.00 +0.00 +0.00 +0.00 ]


Todas as demais variáveis foram inteiras e portanto terão seus valores afixados
Isso produz o seguinte subprograma inteiro para a segunda etapa do RENS:

z = min +1.00 y_1 +1.00 y_2 +1.00 y_3 +1.00 y_4 +1.00 y_5

Subject to:
+1.00 x_1_1 +1.00 x_1_2 +1.00 x_1_3 +1.00 x_1_4 +1.00 x_1_5 = 1.00
+1.00 x_2_1 +1.00 x_2_2 +1.00 x_2_3 +1.00 x_2_4 +1.00 x_2_5 = 1.00
+1.00 x_3_1 +1.00 x_3_2 +1.00 x_3_3 +1.00 x_3_4 +1.00 x_3_5 = 1.00
+1.00 x_4_1 +1.00 x_4_2 +1.00 x_4_3 +1.00 x_4_4 +1.00 x_4_5 = 1.00
+1.00 x_5_1 +1.00 x_5_2 +1.00 x_5_3 +1.00 x_5_4 +1.00 

### letra b) - RINS

In [115]:
# x_tilde
_ = np.vstack((
    [1, 1, 1, 1, 1],
    np.eye(5)
))

print('Solução incumbente do rins: ')
print(_)
print()

_ = _.ravel() # make it 1D
x_tilde = np.zeros(A.shape[1])
x_tilde[0:len(_)] = _

factivel, z, x, J, x_J, A_prime, b_prime, c_prime = rins(A, b, c, x_tilde)

print('Ao executar o RINS, as variáveis afixadas são:')

for j, v in zip(J, x_J):
    print(f'{variables_names.split(',')[j]} = {v:.2f}')

print('O vetor solução de caixas para a construção linearmente relaxada é: ')
_, x_lr, z_lr = resolve_PPL(A, b, c)
print(f'y = {x_lr[0:5]}')

print('O vetor solução de itens em cada caixa, linearmente relaxado, é: ')
result = ''
for i in range(5):
    result += '[ '
    for j in range(5):
        result += f'{x_lr[5 + 5*i + j]:+.2f} '
    result += ']\n'
print(result)

new_variables_names = variables_names + ',' + ','.join(f'n_{i+1:02d}' for i in range(A_prime.shape[1] - A.shape[1]))

print()

print('Temos o seguinte subprograma inteiro como segunda etapa do RINS:\n')
print(
    repr_LP_problem(A_prime, b_prime, c_prime, new_variables_names)
)

print('O resultado da resolução do subprograma inteiro após a aplicação do RINS é: ')

print(f'y = {x[0:5]}')

print('O vetor solução de itens em cada caixa do subprograma inteiro é: ')
result = ''
for i in range(5):
    result += '[ '
    for j in range(5):
        result += f'{x[5 + 5*i + j]:+.2f} '
    result += ']\n'
print(result)

Solução incumbente do rins: 
[[1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]

Ao executar o RINS, as variáveis afixadas são:
y_2 = 1.00
y_3 = 1.00
x_1_2 = 0.00
x_1_4 = -0.00
x_1_5 = 0.00
x_2_1 = 0.00
x_2_2 = 1.00
x_2_3 = 0.00
x_2_4 = 0.00
x_2_5 = 0.00
x_3_1 = 0.00
x_3_2 = 0.00
x_3_3 = 1.00
x_3_4 = 0.00
x_3_5 = 0.00
x_4_1 = 0.00
x_4_5 = -0.00
x_5_1 = -0.00
x_5_3 = 0.00
x_5_4 = 0.00
s_1 = 0.00
s_2 = 0.00
s_3 = 0.00
s_4 = 0.00
s_5 = 0.00
O vetor solução de caixas para a construção linearmente relaxada é: 
y = [0. 1. 1. 0. 0.]
O vetor solução de itens em cada caixa, linearmente relaxado, é: 
[ +0.00 +0.00 +1.00 -0.00 +0.00 ]
[ +0.00 +1.00 +0.00 +0.00 +0.00 ]
[ +0.00 +0.00 +1.00 +0.00 +0.00 ]
[ +0.00 +0.75 +0.25 +0.00 -0.00 ]
[ -0.00 +1.00 +0.00 +0.00 +0.00 ]


Temos o seguinte subprograma inteiro como segunda etapa do RINS:

z = min +1.00 y_1 +1.00 y_2 +1.00 y_3 +1.00 y_4 +1.00 y_5

Subject to:
+1.00 x_1_1 +1.00 x_1_2 +1.00 x_1_3

### letra c)

Considerando um particionamento à minha escolha e a solução incumbente proposta, escolho de fato afixar os valores não nulos da solução incumbente pela conveniẽncia da resolução.

Não vai alterar nada, pois, essa afixação da solução incumbente que é factível não permite quaisquer modificações nos valores das demais variáveis que minimize o problema em tela.

Se eu fizesse um particionamento menor, eu iria ter multiplas formulações de subproblemas para cada particionamento e iria ficar gigante a minha resposta.

## Exercício 2

![](https://i.imgur.com/Bp54CW9.png)

$\bar{x} = [0.44, 1, 0.11, 1]$

### letra a)

Aplicando RENS considerando a solução relaxada $\bar{x}$ afixaríamos $x_2 = x_4 = 1$

Isso resultaria no seguinte subproblema:

$$
\begin{align}
    & \text{max}\ z & = 6 x_1 + 5 x_3 + 7 \\
    & \text{s. t.}\ \\
    & & 5 x_1 + 7 x_3 \leq 3 \\
    & & 4 x_1 + 2 x_3 \leq 2
\end{align}
$$

E a solução obtida é obrigatoriamente $x^{*} = [0, 1, 0, 1], \ z^{*} = 7$

### letra b)

A distância de hamming entre dois vetores binários refere-se ao número de posições em que estes dois vetores se diferem.

Se considerarmos uma distância de Hamming de 1, teremos que comutar um único bit da solução e não podemos comutar 1 e 3 pois, isso torna a solução inviável.

Nesse caso, opta-se por comutar o bit $x_4$  para causar a menor diminuição possível na função objetivo e nesse caso, a melhor solução possível é $x^{*} = [0, 1, 0, 0], \ z^{*} = 4$

### letra c)

Agora podemos comutar até dois bits.

Ligar um e desligar outro, por exemplo.

A melhor solução que pode ser encontrada para a vizinhança de tamanho 2 da solução gerada pelo RENS é:

$$
\begin{align}
x^{*} = [1, 1, 0, 0] \\
z^{*} = 10
\end{align}
$$

Obtida pela comutação dos bits 4 e 2 da solução obtida no RENS